In [ ]:
# Import required packages
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from shutil import copy,move
from distutils.dir_util import copy_tree


In [ ]:
convert={"quad16qubit":'Quadratic',"lin16qubit":"Linear","sin16qubit":"Sine"}

In [ ]:
baseline=[]
for i in glob('function-calc-test/sixteenqubit_test/*/IQP_Full-Pauli-CRX/IQP_Full-Pauli-CRX_results.json'):
    with open(i,'r') as f:
        result = json.load(f)
    baseline.append(pd.DataFrame.from_dict(result).rename(index={0:convert[i.split('/')[2]]})[['R2_train','R2_test']])
baseline=pd.concat(baseline).rename(columns={'R2_train':"Train (baseline)",'R2_test':"Test (baseline)"})

In [ ]:
quantum_linear=[]
for i in glob('function-calc-test/5RUD_5AL/linear/*/*/*results.json'):
    with open(i,'r') as f:
        result = json.load(f)
    quantum_linear.append(pd.DataFrame.from_dict(result).rename(index={0:float(i.split('/')[3])})[['R2_train','R2_test']])
quantum_linear=pd.concat(quantum_linear).reset_index().rename(columns={'R2_train':"Train (quantum)",'R2_test':"Test (quantum)",'index':'ratio'}).melt(id_vars=['ratio']).astype(dict(zip(['ratio', 'variable', 'value'],[float,str,float])))

In [ ]:
quantum_quadratic=[]
for i in glob('function-calc-test/5RUD_5AL/quadratic/*/*/*results.json'):
    with open(i,'r') as f:
        result = json.load(f)
    quantum_quadratic.append(pd.DataFrame.from_dict(result).rename(index={0:float(i.split('/')[3])})[['R2_train','R2_test']])
quantum_quadratic=pd.concat(quantum_quadratic).reset_index().rename(columns={'R2_train':"Train (quantum)",'R2_test':"Test (quantum)",'index':'ratio'}).melt(id_vars=['ratio']).astype(dict(zip(['ratio', 'variable', 'value'],[float,str,float])))

In [ ]:
quantum_sine=[]
for i in glob('function-calc-test/5RUD_5AL/sine/*/*/*results.json'):
    with open(i,'r') as f:
        result = json.load(f)
    quantum_sine.append(pd.DataFrame.from_dict(result).rename(index={0:float(i.split('/')[3])})[['R2_train','R2_test']])
quantum_sine=pd.concat(quantum_sine).reset_index().rename(columns={'R2_train':"Train (quantum)",'R2_test':"Test (quantum)",'index':'ratio'}).melt(id_vars=['ratio']).astype(dict(zip(['ratio', 'variable', 'value'],[float,str,float])))

In [ ]:
# quantum_linear=pd.read_csv('function-calc-test/best_learning_curves/linear/results.csv').rename(columns={'R2_train':"Train (quantum)",'R2_test':"Test (quantum)"}).melt(id_vars=['ratio'])
# quantum_quadratic=pd.read_csv('function-calc-test/best_learning_curves/quadratic/results.csv').rename(columns={'R2_train':"Train (quantum)",'R2_test':"Test (quantum)"}).melt(id_vars=['ratio'])
# quantum_sine=pd.read_csv('function-calc-test/best_learning_curves/sine/results.csv').rename(columns={'R2_train':"Train (quantum)",'R2_test':"Test (quantum)"}).melt(id_vars=['ratio'])

In [ ]:
save=True

In [ ]:
function=['linear','quadratic','sine']
trainsize=[ 0.1,0.3, 0.5, 0.7,0.8]
classical_models=['ridge', 'lasso', 'elastic', 'knn', 'rfr', 'grad', 'svr', 'krr', 'gpr'
]

In [ ]:
classical_results={}
for fn in function:
    classical_results[fn]={}
    for models in classical_models:
        classical_results[fn][models]={}
        for ts in trainsize:
            with open(f"classical_testing/{fn}/{ts}_{models}_scores.json",'r') as f:
                classical_results[fn][models][ts]=json.load(f)
            

In [ ]:
avgs=pd.DataFrame.from_dict({k:{k1:pd.DataFrame.from_dict(v1).loc[['R2_train','R2_test']].mean().mean() for k1,v1 in v.items()} for k,v in classical_results.items()    })

In [ ]:
avgs.T.mean().sort_values()

In [ ]:
palette=sns.color_palette('Paired',4)

In [ ]:
classical_linear=pd.DataFrame.from_dict(classical_results['linear']['svr']).loc[['R2_train','R2_test']].T.rename(columns={'R2_train':"Train (classical)",'R2_test':"Test (classical)"}).reset_index().rename(columns={'index':'ratio'}).melt(id_vars=['ratio']).astype(dict(zip(['ratio', 'variable', 'value'],[float,str,float])))
classical_quadratic=pd.DataFrame.from_dict(classical_results['quadratic']['svr']).loc[['R2_train','R2_test']].T.rename(columns={'R2_train':"Train (classical)",'R2_test':"Test (classical)"}).reset_index().rename(columns={'index':'ratio'}).melt(id_vars=['ratio']).astype(dict(zip(['ratio', 'variable', 'value'],[float,str,float])))
classical_sine=pd.DataFrame.from_dict(classical_results['sine']['svr']).loc[['R2_train','R2_test']].T.rename(columns={'R2_train':"Train (classical)",'R2_test':"Test (classical)"}).reset_index().rename(columns={'index':'ratio'}).melt(id_vars=['ratio']).astype(dict(zip(['ratio', 'variable', 'value'],[float,str,float])))

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 14
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

sns.lineplot(data=quantum_linear,x='ratio',y='value',hue='variable',palette=palette[0:2])
sns.lineplot(data=classical_linear,x='ratio',y='value',hue='variable',palette=palette[2:])
plt.hlines(baseline.loc['Linear'].loc['Train (baseline)'],0,2,color='k',linestyles='--',label='Train (baseline)')
plt.hlines(baseline.loc['Linear'].loc['Test (baseline)'],0,2,color='k',linestyles=':',label='Test (baseline)')

plt.ylabel("R$^{2}$")
plt.xlabel('Ratio of Training Set')
plt.xticks(plt.xticks()[0], ['{:,.0%}'.format(x) for x in plt.xticks()[0]])
plt.title("IQP_Full-Pauli-CRX: Linear")
plt.xlim(0,.91)
plt.ylim(0,1.1)
plt.legend()
plt.tight_layout()
if save:
    plt.savefig('images/Linear_learning_curve.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 14
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
sns.lineplot(data=quantum_quadratic,x='ratio',y='value',hue='variable',palette=palette[0:2])
sns.lineplot(data=classical_quadratic,x='ratio',y='value',hue='variable',palette=palette[2:])
plt.hlines(baseline.loc['Quadratic'].loc['Train (baseline)'],0,2,color='k',linestyles='--',label='Train (baseline)')
plt.hlines(baseline.loc['Quadratic'].loc['Test (baseline)'],0,2,color='k',linestyles=':',label='Test (baseline)')

plt.ylabel("R$^{2}$")
plt.xlabel('Ratio of Training Set')
# plt.xticks(plt.xticks()[0], ['{:,.0%}'.format(x) for x in plt.xticks()[0]])
plt.title("IQP_Full-Pauli-CRX: Quadratic")
plt.xlim(0,.91)
plt.ylim(0,1.1)
plt.legend()
plt.tight_layout()
if save:
    plt.savefig('images/Quadratic_learning_curve.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 14
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
sns.lineplot(data=quantum_sine,x='ratio',y='value',hue='variable',palette=palette[0:2])
sns.lineplot(data=classical_sine,x='ratio',y='value',hue='variable',palette=palette[2:])
plt.hlines(baseline.loc['Sine'].loc['Train (baseline)'],0,2,color='k',linestyles='--',label='Train (baseline)')
plt.hlines(baseline.loc['Sine'].loc['Test (baseline)'],0,2,color='k',linestyles=':',label='Test (baseline)')

plt.ylabel("R$^{2}$")
plt.xlabel('Ratio of Training Set')
# plt.xticks(plt.xticks()[0], ['{:,.0%}'.format(x) for x in plt.xticks()[0]])
plt.title("IQP_Full-Pauli-CRX: Sine")
plt.xlim(0,.91)
plt.ylim(0,1.1)
plt.legend(loc=4)
plt.tight_layout()
if save:
    plt.savefig('images/Sine_learning_curve.png',dpi=300,bbox_inches='tight')
plt.show()